In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD, PCA, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline, make_union

In [2]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 10000)

In [3]:
with open('train_iI26erW/train.csv', 'r') as fp, open('test_plBmD8c.csv') as fg:
    train = pd.read_csv(fp)
    test = pd.read_csv(fg)

/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,21,23,26,28,29,30,32,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,26,28,29,30,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
misCols = train.isnull().sum(axis=0)/train.shape[0]

toremove = []

for k, v in misCols.iteritems():
    if v >= 0.9:
        toremove.append(k)
        
train.drop(toremove, inplace=True, axis=1)
test.drop(toremove, inplace=True, axis=1)

In [7]:
train.fillna(-1, inplace=True)
test.fillna(-1, inplace=True)

In [14]:
for c in train.columns:
    if train[c].dtype == 'object' and c not in ['CUSTOMER_ID','RESPONDERS']:
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

In [15]:
train.head()

,CUSTOMER_ID,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,CC_HOLD_MON_03,CC_HOLD_MON_02,CC_HOLD_MON_01,CC_ACTIVE_MON_03,CC_ACTIVE_MON_02,CC_ACTIVE_MON_01,CC_HOLD,CC_ACTIVE,DC_HOLD,DC_ACTIVE_MON_03,DC_ACTIVE_MON_02,DC_ACTIVE_MON_01,DC_ACTIVE,DC_SPEND_MON_03,DC_SPEND_MON_02,DC_SPEND_MON_01,DC_SPEND_MON_04,CR_LIMIT,STMT_MON_03,STMT_MON_02,STMT_MON_01,COC_ELIGIBLE,EOP_BAL_MON_01,AMB_MON_04,AMB_MON_01,AMB_MON_02,AMB_MON_03,NB_MON_01_CNT,NB_MON_02_CNT,NB_MON_03_CNT,CC_TXN_MON_01,CC_SPEND_MON_06,CC_TXN_MON_06,CC_TXN_MON_03,CC_TXN_MON_02,CC_TXN_MON_04,CC_SPEND_MON_05,CC_TXN_MON_05,DC_TXN_MON_04,DC_TXN_MON_01,DC_SPEND_MON_06,DC_TXN_MON_06,DC_SPEND_MON_05,DC_TXN_MON_05,DC_TXN_MON_03,DC_TXN_MON_02,DC_SPEND_MON_04_GRC,DC_SPEND_MON_04_RST,FINANCE_MON_01,JOBS_MON_01,LIFESTYLE_MON_01,INMON_01KET_MON_01,TRAVEL_MON_01,CHANNEL_CLICK_DISP,DESIGNATION_FINAL,TOP14_CITY,TOP9_CITY,PL2CC_ELIGIBLE,D_AMT_L3_MON_06,C_AMT_L3_MON_06,C_COUNT_L3_MON_06,D_COUNT_L3_MON_06,MAX_C_AMT_L3_MON_06,D_AMT_L3_MON_05,C_AMT_L3_MON_05,C_COUNT_L3_MON_05,D_COUNT_L3_MON_05,MAX_C_AMT_L3_MON_05,D_AMT_L3_MON_04,C_AMT_L3_MON_04,C_COUNT_L3_MON_04,D_COUNT_L3_MON_04,MAX_C_AMT_L3_MON_04,D_AMT_L3_MON_03,C_AMT_L3_MON_03,C_COUNT_L3_MON_03,D_COUNT_L3_MON_03,MAX_C_AMT_L3_MON_03,D_AMT_L3_MON_02,C_AMT_L3_MON_02,C_COUNT_L3_MON_02,D_COUNT_L3_MON_02,MAX_C_AMT_L3_MON_02,D_AMT_L3_MON_01,C_AMT_L3_MON_01,C_COUNT_L3_MON_01,D_COUNT_L3_MON_01,MAX_C_AMT_L3_MON_01,PA_PQ_TAG,NEFT_CC_AMOUNT,NEFT_CC_TXN,NEFT_CC_CATEGORY,NEFT_DC_AMOUNT,NEFT_DC_TXN,NEFT_DC_CATEGORY,TPT_DC_CATEGORY_MON_01,TPT_CC_CATEGORY_MON_01,IMPS_CC_CATEGORY_MON_01,ZIP_CODE_FINAL,CRED_NEED_SCORE,CARD_AUTOMOBILE_MON_01,CARD_HOTEL_MON_01,CARD_JEWELLERY_MON_01,CARD_RESTAURANT_MON_01,CARD_TRAVEL_MON_01,CARD_PRSNL_CARE_MON_01,CARD_HOBBY_MON_01,CARD_HOME_DECOR_MON_01,CARD_MEDICAL_MON_01,CARD_ENTMNT_MON_01,SAL_MON_01,SAL_MON_02,SAL_MON_03,TOP_CORP_TAG,EOP_MON_02,EOP_MON_03,CREDIT_ACT,DEBIT_ACT,SCRUB_EMI,RESPONDERS
0,385793,0,2,2,0,21,0.00,0.00,0,0,0,0,0,0,0,0,1,0,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,0,0.00,0.00,0.00,0.00,0.00,-1.0,-1.0,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0,0,0,2,4,1,1,0,39.00,-1.00,0.0,1.0,-1.0,-1.00,-1.0,-1.0,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.0,-1.0,-1.0,-1.00,-1.00,-1.0,-1.0,-1.0,-1.00,-1.00,-1.0,-1.0,-1.0,0,-1.0,-1.0,13,-1.0,-1.0,13,13,13,13,9960,-1.000000,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0.00,0.00,0,0,0.000000,N
1,286891,0,2,2,0,23,8243.77,7.77,0,0,0,0,0,0,0,0,1,1,1,1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,0,11.77,1417.26,300.51,593.06,418.09,-1.0,-1.0,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0,-1.0,-1.00,-1.0,8.0,5.0,504.0,7.0,631.0,13.0,12.0,1.0,-1.0,-1.0,0,0,0,0,0,2,4,1,1,0,17250.00,17870.00,3.0,19.0,7957.0,12965.50,12336.5,6.0,21.0,8011.0,23448.52,23457.0,4.0,12.0,12000.0,17161.73,17154.75,6.0,20.0,8144.0,8720.00,11733.75,4.0,3.0,7519.0,13763.00,10758.00,3.0,9.0,8240.0,0,-1.0,-1.0,13,-1.0,-1.0,13,13,13,13,19110,-1.000000,0,0,0,0,0,0,0,0,0,0,10754.0,8733.0,17154.0,0,3016.77,3.02,1,1,0.000000,N
2,108751,0,2,2,0,41,26515.00,0.00,0,0,0,0,0,0,0,0,1,0,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,0,26458.25,5354.65,6100.73,7955.82,11138.53,1.0,1.0,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0,-1.0,-1.00,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0,0,1,2,4,0,0,0,39362.25,39125.00,2.0,8.0,28171.0,2005.75,31261.0,3.0,2.0,16171.0,29064.00,16197.0,2.0,3.0,16171.0,8727.25,7180.00,2.0,3.0,4680.0,36375.75,31516.00,2.0,4.0,16516.0,28520.75,44980.00,5.0,3.0,16516.0,0,-1.0,-1.0,13,26515.0,1.0,12,13,13,13,298,-1.000000,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,9999.00,14858.75,1,1,0.000000,N
3,278651,0,2,2,0,28,65917.28,8270.28,1,1,1,1,0,1,1,1,1,1,1,1,1,10960.0,795.0,-1.0,5340.0,62000.0,6051.22,22681.82,0.00,1,65917.28,113120.39,35560.81,42686.93,61593.38,1.0,1.0,1.0,3.0,10935.00,5.0,15.0,-1.0,9.0,803.00,4.0,2.0,6.0,-1.0,-1.0,1531.0,2.0,9.0,12.0,1330.0,-1.0,1,0,0,0,0,2,4,0,0,0,128169.25,20152.03,2.0,20.0,19

In [26]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

In [31]:
train['RESPONDERS'] = train['RESPONDERS'].replace(to_replace = {'Y':1, 'N':0})
target = train['RESPONDERS']

In [33]:
feature_names = [x for x in train.columns if x not in ['CUSTOMER_ID','RESPONDERS']]

In [83]:
nfolds = 7
skf = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=9087)

In [68]:
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1,
          "max_depth": 4,
          "subsample": 0.9,
          "colsample_bytree": 0.6,
          "min_child_weight": 2*3,
          "seed": 2016, 
          "tree_method": "exact",
         "eval_metric":"auc"}

In [82]:
oof_train = pd.DataFrame({'CUSTOMER_ID':train['CUSTOMER_ID'], 'RESPONDERS':0})
allpredictions = pd.DataFrame()
score = []

In [84]:
increase = True
for i, (train_index, test_index) in enumerate(skf.split(train, target)):
    print('Fold %d/%d'%(i+1, nfolds))
    X_train, X_valid = train.iloc[train_index], train.iloc[test_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[test_index]
    
    if increase:
        pos = pd.Series(target == 1)

        X_train = pd.concat([X_train, X_train[pos]], axis=0)
        y_train = pd.concat([y_train, y_train.loc[pos]], axis=0)
        
        idx = np.arange(len(X_train))
        np.random.shuffle(idx)
        X_train = X_train.iloc[idx]
        y_train = y_train.iloc[idx]
    
    dtrain = xgb.DMatrix(X_train[feature_names], label=y_train, missing=-1)
    dvalid = xgb.DMatrix(X_valid[feature_names], label=y_valid, missing = -1)
    
    watchlist = [(dtrain, 'train'),(dvalid,'valid')]
    
    clf = xgb.train(params,
                    dtrain,
                    num_boost_round=10000,
                    evals=watchlist,
                    early_stopping_rounds=40,
                    verbose_eval=20,
                    maximize = True)
    
    pred1 = clf.predict(dvalid)
    oof_train.loc[test_index, 'RESPONDERS'] = pred1
    
    scr = roc_auc_score(y_valid, pred1)
    dtest = xgb.DMatrix(test[feature_names], missing= - 1)
    preds2 = clf.predict(dtest)
    
    allpredictions['p'+str(i)] = preds2
    score.append(scr)
    
    del X_train, X_valid, y_train, y_valid, clf, pred1

Fold 1/7


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


[0]	train-auc:0.816296	valid-auc:0.818877
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[20]	train-auc:0.864038	valid-auc:0.858376
[40]	train-auc:0.873	valid-auc:0.864849
[60]	train-auc:0.883539	valid-auc:0.870516
[80]	train-auc:0.891555	valid-auc:0.873721
[100]	train-auc:0.897471	valid-auc:0.875445
[120]	train-auc:0.902283	valid-auc:0.876385
[140]	train-auc:0.906198	valid-auc:0.876962
[160]	train-auc:0.910037	valid-auc:0.877803
[180]	train-auc:0.913406	valid-auc:0.878202
[200]	train-auc:0.917493	valid-auc:0.87854
[220]	train-auc:0.920292	valid-auc:0.878074
[240]	train-auc:0.923201	valid-auc:0.878276
Stopping. Best iteration:
[201]	train-auc:0.9176	valid-auc:0.878599

Fold 2/7
[0]	train-auc:0.816268	valid-auc:0.814429
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[20]	train-auc:0.864213	valid-au

In [85]:
np.mean(score)

0.87486859626379154

In [87]:
oof_train.to_csv('submissions/oof_preds/xgb_train1.csv', index=False)

In [88]:
allpredictions.head()

,p0,p1,p2,p3,p4,p5,p6
0,0.010397,0.009606,0.009854,0.011680,0.007875,0.006835,0.016271
1,0.013806,0.010962,0.016313,0.011847,0.025325,0.015108,0.021205
2,0.001101,0.001163,0.001230,0.001291,0.001350,0.001239,0.001251
3,0.001208,0.001245,0.001334,0.001174,0.001164,0.001158,0.001206
4,0.025599,0.020299,0.031069,0.028825,0.015245,0.033325,0.023601


In [89]:
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = allpredictions.mean(axis=1).values

In [90]:
sub.to_csv('submissions/xgb_bagging_seven.csv', index=False) # 0.58

### Do PCA and get new features

In [93]:
dim_train = train[feature_names]
dim_test = test[feature_names]

In [94]:
from sklearn.preprocessing import RobustScaler

In [95]:
rb = RobustScaler()

In [101]:
dim_train = rb.fit_transform(dim_train)
dim_test = rb.fit_transform(dim_test)

In [91]:
n_comp = 5

In [105]:
tsvd = TruncatedSVD(n_components=n_comp, random_state=42)
tsvd_train = tsvd.fit_transform(dim_train)
tsvd_test = tsvd.transform(dim_test)

In [107]:
pca = PCA(n_components=n_comp, random_state=420)
pca_train = pca.fit_transform(dim_train)
pca_test = pca.transform(dim_test)

In [108]:
ica = FastICA(n_components=n_comp, random_state=2030)
ica_train = ica.fit_transform(dim_train)
ica_test = ica.transform(dim_test)

In [109]:
grp = GaussianRandomProjection(n_components=n_comp, random_state=42)
grp_train = grp.fit_transform(dim_train)
grp_test = grp.transform(dim_test)

In [110]:
srp = SparseRandomProjection(n_components=n_comp, random_state=42)
srp_train = srp.fit_transform(dim_train)
srp_test = srp.transform(dim_test)

In [111]:
for i in range(1, n_comp + 1):
    train['pca_' + str(i)] = pca_train[:,i-1]
    test['pca_' + str(i)] = pca_test[:,i-1]
    
    train['tsvd_' + str(i)] = tsvd_train[:,i-1]
    test['tsvd_' + str(i)] = tsvd_test[:,i-1]
    
    train['ica_' + str(i)] = ica_train[:,i-1]
    test['ica_' + str(i)] = ica_test[:,i-1]
    
    train['grp_' + str(i)] = grp_train[:,i-1]
    test['grp_' + str(i)] = grp_test[:,i-1]
    
    train['srp_' + str(i)] = srp_train[:,i-1]
    test['srp_' + str(i)] = srp_test[:,i-1]

In [112]:
train.shape

(300000, 153)

In [113]:
test.shape

(200000, 152)

### XGB 1

In [120]:
from sklearn.model_selection import train_test_split

In [121]:
X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size = 0.3, stratify = target)

In [122]:
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1,
          "max_depth": 4,
          "subsample": 0.9,
          "colsample_bytree": 0.6,
          "min_child_weight": 2*3,
          "seed": 2016, 
          "tree_method": "exact",
         "eval_metric":"auc"}

In [171]:
dtrain = xgb.DMatrix(X_train[feature_names], y_train, missing=-1)
dvalid = xgb.DMatrix(X_valid[feature_names], y_valid, missing=-1)
dtest = xgb.DMatrix(test[feature_names], missing=-1)

In [126]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [127]:
clf1 = xgb.train(params, dtrain, num_boost_round=10000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

[0]	train-auc:0.823539	valid-auc:0.82222
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[20]	train-auc:0.865959	valid-auc:0.858904
[40]	train-auc:0.871839	valid-auc:0.861946
[60]	train-auc:0.881822	valid-auc:0.866571
[80]	train-auc:0.890338	valid-auc:0.86928
[100]	train-auc:0.897192	valid-auc:0.871136
[120]	train-auc:0.902828	valid-auc:0.872336
[140]	train-auc:0.90743	valid-auc:0.873333
[160]	train-auc:0.911658	valid-auc:0.873165
[180]	train-auc:0.916158	valid-auc:0.873491
[200]	train-auc:0.920118	valid-auc:0.873661
[220]	train-auc:0.92325	valid-auc:0.873668
[240]	train-auc:0.926623	valid-auc:0.873039
Stopping. Best iteration:
[208]	train-auc:0.921646	valid-auc:0.8741



In [172]:
preds1 = clf1.predict(dtest)

In [174]:
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = preds1

In [175]:
sub.to_csv('submissions/xgb_single_two.csv', index=False) ## 0.5682

### XGB 2 - remove zero variance features

In [161]:
## remove zero variance columns
var_check = pd.DataFrame(columns=['name','variance'])

for i, c in enumerate(train.columns):
    if c not in ['CUSTOMER_ID','RESPONDERS']:
        var_check.loc[i, 'name'] = c
        var_check.loc[i, 'variance'] = train[c].var()
        
var_check.sort_values('variance', ascending=True).reset_index(drop=True)

In [169]:
cols_to_use = var_check[var_check['variance'] > 1]['name'].tolist() ## remove columns with variance lesser than zero

In [176]:
params = {"objective": "binary:logistic",
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.1,
          "max_depth": 5,
          "subsample": 0.8,
          "colsample_bytree": 0.3,
          "min_child_weight": 0,
          "seed": 2017, 
          "tree_method": "exact",
         "eval_metric":"auc"}

In [177]:
dtrain = xgb.DMatrix(X_train[cols_to_use], y_train, missing=-1)
dvalid = xgb.DMatrix(X_valid[cols_to_use], y_valid, missing=-1)
dtest = xgb.DMatrix(test[cols_to_use], missing=-1)

In [178]:
watchlist = [(dtrain, 'train'),(dvalid, 'valid')]

In [179]:
clf1 = xgb.train(params, dtrain, num_boost_round=10000, evals=watchlist, maximize=True, verbose_eval=20, early_stopping_rounds=40)

[0]	train-auc:0.785563	valid-auc:0.779417
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[20]	train-auc:0.869232	valid-auc:0.852376
[40]	train-auc:0.884828	valid-auc:0.862336
[60]	train-auc:0.897585	valid-auc:0.866906
[80]	train-auc:0.909083	valid-auc:0.86894
[100]	train-auc:0.918052	valid-auc:0.870188
[120]	train-auc:0.925934	valid-auc:0.87186
[140]	train-auc:0.931686	valid-auc:0.871643
[160]	train-auc:0.937222	valid-auc:0.871625
Stopping. Best iteration:
[127]	train-auc:0.928259	valid-auc:0.872028



In [180]:
preds1 = clf1.predict(dtest)

In [181]:
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = preds1

In [182]:
sub.to_csv('submissions/xgb_single_three.csv', index=False) ## 54 - score dropping.

### XGB BAGGED

In [117]:
oof_train = pd.DataFrame({'CUSTOMER_ID':train['CUSTOMER_ID'], 'RESPONDERS':0})
allpredictions = pd.DataFrame()
score = []

In [118]:
increase = True
for i, (train_index, test_index) in enumerate(skf.split(train, target)):
    print('Fold %d/%d'%(i+1, nfolds))
    X_train, X_valid = train.iloc[train_index], train.iloc[test_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[test_index]
    
    if increase:
        pos = pd.Series(target == 1)

        X_train = pd.concat([X_train, X_train[pos]], axis=0)
        y_train = pd.concat([y_train, y_train.loc[pos]], axis=0)
        
        idx = np.arange(len(X_train))
        np.random.shuffle(idx)
        X_train = X_train.iloc[idx]
        y_train = y_train.iloc[idx]
    
    dtrain = xgb.DMatrix(X_train[feature_names], label=y_train, missing=-1)
    dvalid = xgb.DMatrix(X_valid[feature_names], label=y_valid, missing = -1)
    
    watchlist = [(dtrain, 'train'),(dvalid,'valid')]
    
    clf = xgb.train(params,
                    dtrain,
                    num_boost_round=10000,
                    evals=watchlist,
                    early_stopping_rounds=40,
                    verbose_eval=20,
                    maximize = True)
    
    pred1 = clf.predict(dvalid)
    oof_train.loc[test_index, 'RESPONDERS'] = pred1
    
    scr = roc_auc_score(y_valid, pred1)
    dtest = xgb.DMatrix(test[feature_names], missing= - 1)
    preds2 = clf.predict(dtest)
    
    allpredictions['p'+str(i)] = preds2
    score.append(scr)
    
    del X_train, X_valid, y_train, y_valid, clf, pred1

Fold 1/7


/home/manish/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


[0]	train-auc:0.818001	valid-auc:0.812239
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[20]	train-auc:0.86827	valid-auc:0.862023
[40]	train-auc:0.8741	valid-auc:0.865919
[60]	train-auc:0.880807	valid-auc:0.869118
[80]	train-auc:0.889309	valid-auc:0.873141
[100]	train-auc:0.896446	valid-auc:0.874864
[120]	train-auc:0.901552	valid-auc:0.876413
[140]	train-auc:0.906566	valid-auc:0.877953
[160]	train-auc:0.911221	valid-auc:0.878617
[180]	train-auc:0.915605	valid-auc:0.879051
[200]	train-auc:0.919098	valid-auc:0.878762
[220]	train-auc:0.922772	valid-auc:0.878656
Stopping. Best iteration:
[184]	train-auc:0.91632	valid-auc:0.879062

Fold 2/7
[0]	train-auc:0.81847	valid-auc:0.815435
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 40 rounds.
[20]	train-auc:0.867088	valid-auc:0.866811
[40]	train-auc:0.87394	valid-auc

KeyboardInterrupt: 

In [85]:
np.mean(score)

0.87486859626379154

In [87]:
oof_train.to_csv('submissions/oof_preds/xgb_train1.csv', index=False)

In [88]:
allpredictions.head()

,p0,p1,p2,p3,p4,p5,p6
0,0.010397,0.009606,0.009854,0.011680,0.007875,0.006835,0.016271
1,0.013806,0.010962,0.016313,0.011847,0.025325,0.015108,0.021205
2,0.001101,0.001163,0.001230,0.001291,0.001350,0.001239,0.001251
3,0.001208,0.001245,0.001334,0.001174,0.001164,0.001158,0.001206
4,0.025599,0.020299,0.031069,0.028825,0.015245,0.033325,0.023601


In [89]:
sub = pd.read_csv('sample_submission_ROqqAAN.csv')
sub['CUSTOMER_ID'] = test['CUSTOMER_ID']
sub['RESPONDERS'] = allpredictions.mean(axis=1).values

In [90]:
sub.to_csv('submissions/xgb_bagging_seven.csv', index=False) # 0.58